In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
#import skimage.registration as reg

In [2]:
import ar
import invariants

In [8]:
import cv2


def read_avi_file(path: str) -> np.ndarray:
    """Reads an .avi file and returns a stack of frames."""
    cap = cv2.VideoCapture(path)
    frames = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))  # Convert BGR to RGB

    cap.release()
    return np.array(frames)

vid = read_avi_file(path=r"D:\chp\1001-8.avi")

In [10]:
vid.shape

(1164, 480, 640)

In [4]:
# Read in a video.
# vid = np.load("../data/wavy.npy")
vid = read_avi_file(path = r"D:\chp\1001-8.avi")
print(vid.shape)

ValueError: maximum supported dimension for an ndarray is 32, found 1164

In [ ]:
plt.imshow(vid[0], cmap = "gray")

In [ ]:
# Compute optical flow.
import cv2

In [ ]:
#cv2.calcOpticalFlowFarneback(
#  frame1,
#  frame2,
#  prevFlow,
#  pyr_scale = 0.5,
#  levels = 3 (number of pyramid levels),
#  winsize = 30 (larger = more robust but also blurrier),
#  iterations = 10 (number of iterations at each layer),
#  poly_n = 5 (larger is more robust but blurrier),
#  poly_sigma = 1.1 (size of gaussian blur),
#  flags = cv2.OPTFLOW_USE_INITIAL_FLOW & cv2.OPTFLOW_FARNEBACK_GAUSSIAN
# )

#opt = cv2.calcOpticalFlowFarneback(vid[0], vid[1], np.zeros(shape = vid[0].shape), 0.5, 3, 30, 10, 7, 1.5, cv2.OPTFLOW_FARNEBACK_GAUSSIAN)

In [11]:
def draw_flow(img, flow, step=6):
    h, w = img.shape[:2]
    y, x = np.mgrid[step/2:h:step, step/2:w:step].reshape(2,-1).astype(int)
    fx, fy = flow[y,x].T
    lines = np.vstack([x, y, x+fx, y+fy]).T.reshape(-1, 2, 2)
    lines = np.int32(lines + 0.5)
    vis = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    cv2.polylines(vis, lines, 0, (0, 255, 0))
    for (x1, y1), (_x2, _y2) in lines:
        cv2.circle(vis, (x1, y1), 1, (0, 255, 0), -1)
    return vis

In [12]:
from ipywidgets import interact  # Where the magic happens with ipywidgets.
from PIL import Image
from tqdm import tqdm

In [13]:
def display_video(video):
    n_frames = video.shape[0]
    
    # This is our callback function, and what makes the widget possible.
    def _show(frame = 1):
        return Image.fromarray(video[frame - 1])
    
    return interact(_show, frame = (1, n_frames))

In [14]:
flow_vid = []
rot = []

prev_flow = None
for i in tqdm(range(1, vid.shape[0])):
    curr = vid[i]
    prev = vid[i - 1]
    
    opt = cv2.calcOpticalFlowFarneback(
        prev,
        curr,
        prev_flow, 
        0.75, # pyr_scale
        7,   # levels
        15,  # winsize
        20,  # iterations
        7,   # poly_n
        0.9, # poly_sigma
        cv2.OPTFLOW_FARNEBACK_GAUSSIAN & cv2.OPTFLOW_USE_INITIAL_FLOW
    )
    opt = np.array(opt, dtype = np.float64)
    rot.append(invariants.curl(opt[:, :, 0], opt[:, :, 1]))
    flow_vid.append(draw_flow(curr, opt))
    prev_flow = opt
flow_vid = np.array(flow_vid)
rot = np.array(rot)

100%|██████████| 1163/1163 [07:51<00:00,  2.47it/s]


In [ ]:
display_video(np.array(flow_vid[:50]))

In [ ]:
bwrot = rot.copy()
bwrot += np.abs(bwrot.min())
bwrot /= bwrot.max()
bwrot *= 256
bwrot = np.array(bwrot, np.uint8)

In [ ]:
display_video(bwrot[:100])

In [ ]:
import scipy.signal

In [ ]:
diff = np.abs(bwrot.max(axis = 0) - bwrot.min(axis = 0))
diff = scipy.signal.medfilt2d(diff, kernel_size = 11)
plt.imshow(diff, cmap = "Blues")

In [ ]:
plt.hist(diff.flatten(), bins = 50)

In [ ]:
patch = rot[:, 125:160, 40:60]
raster = patch.reshape((patch.shape[0], -1))
plt.plot(raster)

In [ ]:
import spq.ar as ar

In [16]:
order = 5

image = np.zeros(shape = (order, rot.shape[1], rot.shape[2]))
for row in tqdm(range(image.shape[1])):
    for col in range(image.shape[2]):
        a = ar.train(rot[:, row, col], order)
        for i in range(image.shape[0]):
            image[i, row, col] = a[i][0][0]

100%|██████████| 480/480 [01:44<00:00,  4.60it/s]


In [ ]:
from utils import raster_scan
X = raster_scan(rot).T
X.shape

In [ ]:
plt.figure(figsize = (16, 12))
for i in range(order):
    plt.subplot(1, order, i + 1)
    plt.title("$a_{}$".format(i + 1))
    plt.imshow(image[i], cmap = "Blues", vmin = image.min(), vmax = image.max())

In [ ]:
_ = plt.hist(image[0].flatten(), bins = 50)

In [ ]:
import sklearn.mixture as skm

In [ ]:
gmm = skm.GaussianMixture(n_components = 2, random_state = 42)
y = gmm.fit_predict(image[0].flatten().reshape(-1, 1))

In [ ]:
plt.imshow(y.reshape(image[0].shape), cmap = "Blues")

In [ ]:
_ = plt.hist(image[1].flatten(), bins = 50)

In [ ]:
gmm = skm.GaussianMixture(n_components = 2, random_state = 42)
y = gmm.fit_predict(image[1].flatten().reshape(-1, 1))

In [ ]:
plt.imshow(y.reshape(image[1].shape), cmap = "Blues")

In [ ]:
_ = plt.hist(image[2].flatten(), bins = 50)

In [ ]:
gmm = skm.GaussianMixture(n_components = 2, random_state = 42)
y = gmm.fit_predict(image[2].flatten().reshape(-1, 1))

In [ ]:
plt.imshow(y.reshape(image[2].shape), cmap = "Blues")

In [ ]:
yb = np.array(y, dtype = np.uint8)
mask = yb.reshape(image[0].shape)

In [ ]:
img = np.zeros((image.shape[1], image.shape[2], 3), dtype = np.uint8)
for index in range(3):
    channel = image[index].copy()
    channel -= channel.min()
    channel /= channel.max()
    channel *= 255
    img[:, :, index] = np.array(channel, dtype = np.uint8)

In [ ]:
fgModel = np.zeros((1, 65), dtype = np.float64)
bgModel = np.zeros((1, 65), dtype = np.float64)

In [ ]:
(mask, bgModel, fgModel) = cv2.grabCut(img, mask, None, bgModel, fgModel, 
                                       iterCount = 200, mode = cv2.GC_INIT_WITH_MASK)

In [ ]:
plt.imshow(mask, cmap = "Blues")

# Let's try a different video

In [ ]:
#np.save("wavy_ar.npy", image)

In [ ]:
# Read in a video.
div = np.load("data/videos/stiff_dyskinetic.npy")
print(div.shape)

In [ ]:
flow_vid = []
drot = []

prev_flow = None
for i in tqdm(range(1, div.shape[0])):
    curr = div[i]
    prev = div[i - 1]
    
    opt = cv2.calcOpticalFlowFarneback(
        prev,
        curr,
        prev_flow, 
        0.85, # pyr_scale
        7,   # levels
        15,  # winsize
        20,  # iterations
        7,   # poly_n
        0.7, # poly_sigma
        cv2.OPTFLOW_FARNEBACK_GAUSSIAN & cv2.OPTFLOW_USE_INITIAL_FLOW
    )
    opt = np.array(opt, dtype = np.float64)
    drot.append(invariants.curl(opt[:, :, 0], opt[:, :, 1]))
    flow_vid.append(draw_flow(curr, opt))
    prev_flow = opt
flow_vid = np.array(flow_vid)
drot = np.array(drot)

In [ ]:
bwrot = drot.copy()
bwrot += np.abs(bwrot.min())
bwrot /= bwrot.max()
bwrot *= 256
bwrot = np.array(bwrot, np.uint8)

In [ ]:
display_video(bwrot[:100])

In [ ]:
dimage = np.zeros(shape = (order, drot.shape[1], drot.shape[2]))
for row in tqdm(range(dimage.shape[1])):
    for col in range(dimage.shape[2]):
        a = ar.train(drot[:, row, col], order)
        dimage[:, row, col] = a

In [ ]:
plt.figure(figsize = (16, 12))
for i in range(order):
    plt.subplot(1, order, i + 1)
    plt.title("$a_{}$".format(i + 1))
    plt.imshow(dimage[i], cmap = "Blues", vmin = dimage.min(), vmax = dimage.max())

In [ ]:
np.save("stiff_dyskinetic_AR.npy", dimage)

In [ ]:
np.save("normal_AR.npy", image)

In [ ]:
np.save("wavy_AR.npy", image)